Goals:
======

- Read an ASDF file
- Use tools for looking at the file and searching for attributes
- Display an image contained in the file
- Access and modify metadata
- Write the file to disk

The file jwst.asdf is presumed to be in your current directory (see xxx for instructions on how to get all files used in this tutorial)

In [ ]:
import numpy as np
import asdf
from matplotlib import pyplot as plt

Load ASDF file
--------------

In [ ]:
af = asdf.open('jwst.asdf')
# Get some info about the contents
af.info()

In [ ]:
# The file has lots of content so some options are needed to see it all
af.info(max_rows=200)

In [ ]:
# Search for attributes with exposure in their name
af.search('exposure')

In [ ]:
# Assign an attribute value to a variable
elapsed_time = af.tree['meta']['exposure']['elapsed_exposure_time']
print(elapsed_time)

In [ ]:
# Both JWST and Roman datamodels add syntactix "sugar" that allow the following syntax
# presuming the file was opened as a datmodel, e.g.,
# dm = jwst.datamodel.open('jwst.asdf')
# print(dm.meta.exposure.elapsed_exposure_time)

In [ ]:
# Now load image and display it
im = af.tree['data']
print(im.shape)
plt.imshow(im)

In [ ]:
# Obviously we need to adjust the stretch on this. I've already found a reasonable one.
plt.clf()
plt.imshow(im, vmin=0, vmax=5)

In [ ]:
# To use a log transfer function means getting rid of all NaNs
nim = np.array(im) # Existing image is only read only and not a numpy array
nim[np.isnan(nim)] = 5
nim[nim<=0] = 1
plt.clf()
plt.imshow(np.log(nim), vmin=0, vmax=2.5)


Modifying the contents
----------------------

In [ ]:
# Arbitrarily change the elapsed_exposure_time
af.tree['meta']['exposure']['elapsed_exposure_time'] = 1800
print(af.tree['meta']['exposure']['elapsed_exposure_time'])

In [ ]:
# Replace data attribute with the log version of the image.
af.tree['data'] = np.log(nim)

In [ ]:
# Create new meta attribute
af.tree['meta']['inspected_by'] = 'Jacques Clouseau'

Saving the modified contents
----------------------------

In [ ]:
# Save to a different file
af.write_to('jwst_modified.asdf')

In [ ]:
ls

Exercise
--------

- Search for the name of the PI ("Klaus") and replace the attribute containing it with your own name. Type "af.search?" to show how to search values instead of attribute names.
- Use the above log image and set all pixels in rows 3000:3010 and columns 4000:4010 to a value of 2.5, and replace the data attribute with the modified log image.
- Save the result to a new file named "jwst_exercise.asdf"
- Load that new file and display the data image
- Print out the value of the attribute that contains the name of the PI to confirm the edits were saved

In [ ]:
af.search(value = "Klaus")

In [ ]:
af.search?

In [ ]:
lim = np.log(nim)
lim[3000:3010, ] = 2.5
lim[:, 4000:4010] = 2.5
plt.clf()
plt.imshow(lim, vmin=0, vmax=2.5)